<a href="https://colab.research.google.com/github/hamzamusht550/Amazon-Clone-Frontend/blob/main/MKV_Combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

#@title Install Dependencies

from IPython.display import clear_output
from google.colab import drive
from google.colab import auth

!apt-get install -y mediainfo ffmpeg mkvtoolnix aria2
!pip install patool pymediainfo ffmpeg-python pyrogram tgcrypto nest_asyncio

drive.mount('/content/drive')
auth.authenticate_user()
clear_output()

In [ ]:

#@title Mediainfo MKV

MKV_INFO = "/content/drive/MyDrive/5.mkv" #@param {type: "string"}

from IPython.display import clear_output
clear_output()

# !ffmpeg -i {MKV_INFO}
!mediainfo "{MKV_INFO}"
!du -h "{MKV_INFO}"
!du -BM "{MKV_INFO}"

In [ ]:

#@title URL Downloader

URLs = "https://brucewayne.tgmkvofcl.workers.dev/217765?hash=AgADFw, https://brucewayne.tgmkvofcl.workers.dev/217767?hash=AgADGw, https://brucewayne.tgmkvofcl.workers.dev/217769?hash=AgADGA, https://brucewayne.tgmkvofcl.workers.dev/217771?hash=AgADHA, https://brucewayne.tgmkvofcl.workers.dev/217776?hash=AgADGg, https://brucewayne.tgmkvofcl.workers.dev/217773?hash=AgADHQ, https://brucewayne.tgmkvofcl.workers.dev/217779?hash=AgADHg, https://brucewayne.tgmkvofcl.workers.dev/217781?hash=AgADHw, https://brucewayne.tgmkvofcl.workers.dev/217783?hash=AgADIg, https://brucewayne.tgmkvofcl.workers.dev/217787?hash=AgADGQ, https://brucewayne.tgmkvofcl.workers.dev/217785?hash=AgADJA, https://brucewayne.tgmkvofcl.workers.dev/217786?hash=AgADIQ, https://brucewayne.tgmkvofcl.workers.dev/217791?hash=AgADJg, https://brucewayne.tgmkvofcl.workers.dev/217792?hash=AgADJw, https://brucewayne.tgmkvofcl.workers.dev/217794?hash=AgADJQ, "  # @param {type: "string"}
OUTPUT_DIR = "/content/6"  #@param {type: "string"}
Select_Downloader = "Aria2"  #@param ["Aria2", "wget"]

def strToList(listStr, separator=","):
    listStr = ' '.join(listStr.split()) # Remove extra spaces from String
    listStr = '\n'.join(listStr.split()) # Remove newLines from String
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    finalList = list(set(myList)) # Remove duplicates items from list
    showAllElements = "\n".join(f"{i+1} ==> {ele}" for i, ele in enumerate(finalList))
    print(showAllElements, "\n\n")
    return finalList

def main(urlStr):
    URLs_List = strToList(urlStr, ",")
    for i, _url in enumerate(URLs_List):
        print(f"\n{i+1} ==> {_url}")
        match Select_Downloader:
            case "Aria2":
                !aria2c --continue=true --file-allocation=none -x 16 -s 16 -d "{OUTPUT_DIR}" "{_url}"
            case "wget":
                !wget -nc -P "{OUTPUT_DIR}" "{_url}"


main(URLs)

In [ ]:

#@title Show MKV tracks

FILES_Folder = "/content/6"  #@param {type: "string"}
Output_HTML = True  #@param {type: "boolean"}

from IPython.display import HTML
from pymediainfo import MediaInfo
import os

def get_track_info(file_path):
    try:
        media_info = MediaInfo.parse(file_path)
        tracks_info = []
        for track in media_info.tracks:
            if track.track_type in ["Video", "Audio", "Text"]:  # Only show relevant tracks
                track_id = track.track_id or "Unknown"
                track_type = track.track_type or "Unknown"
                codec = track.codec_id or "Unknown"
                language = track.language or "Unknown"
                title = track.title or "Untitled"
                tracks_info.append((track_id, track_type, codec, language, title))
        return tracks_info
    except Exception as e:
        print(f"Error occurred:\n{e}")
        return []

def getMKVFilesList(folder_path, relative=0):
    all_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.mkv'):
                full_path = os.path.join(root, file)
                if relative == 1:
                    full_path = os.path.relpath(full_path, folder_path)
                all_files.append(full_path)

    all_files.sort()
    return all_files

def compare_tracks(tracks1, tracks2):
    return tracks1 == tracks2

def main(folder_path, use_html=True):
    files = getMKVFilesList(folder_path)
    if not files:
        print("No MKV files found in the specified folder.")
        return

    reference_tracks = get_track_info(files[0])
    # HTML output with copy-to-clipboard buttons
    html_output = f"""
    <h3 onclick="copyToClipboard('{files[0]}')">Reference file: <b>{files[0]}</b>
    </h3>
    <ul>
    """
    for track in reference_tracks:
        html_output += f"<li>ID: {track[0]} | Type: {track[1]} | Codec: {track[2]} | Language: {track[3]} | Title: {track[4]}</li>"
        html_output += f"<li><b>Complete INFO: {track}</b></li>"
    html_output += "</ul>"

    for i, _File in enumerate(files[1:], start=1):
        current_tracks = get_track_info(_File)
        if reference_tracks != current_tracks:
            html_output += f"""
            <h3 onclick="copyToClipboard('{_File}')" style='color:red;'>
            {i+1} ==> {_File} (Different Tracks)
            </h3>
            """
        if reference_tracks != current_tracks:
         html_output += "<ul>"
         for track in current_tracks:
             html_output += f"<li>ID: {track[0]} | Type: {track[1]} | Codec: {track[2]} | Language: {track[3]} | Title: {track[4]}</li>"
             html_output += f"<li><b>Complete INFO: {track}</b></li>"
         html_output += "</ul>"

    # Add JavaScript for copying to clipboard
    html_output += """
    <script>
    function copyToClipboard(text) {
        navigator.clipboard.writeText(text).then(function() {
            alert('Copied to clipboard: ' + text);
        }, function(err) {
            alert('Failed to copy: ' + err);
        });
    }
    </script>
    """

    display(HTML(html_output))


main(FILES_Folder, Output_HTML)

In [ ]:

#@title MKV Merger

MKV_FILES = "/content/6"  # @param {type: "string"}
OUTPUT_DIR = ""  #@param {type: "string"}
MAX_FileSize = 4  #@param {type: "number"}
JUST_DOWNLOAD= False  #@param {type: "boolean"}
Delete_Extracted_DIR= False  #@param {type: "boolean"}
# Delete_Extracted_Data = False  #@param {type: "boolean"}
# #@markdown ***Start 00:00:00 / End 00:00:10***
# trim_start = ""  #@param {type: "string"}
# trim_end = ""  #@param {type: "string"}

from IPython.display import clear_output
import subprocess
import os
import re
import requests
import patoolib
from google.colab import drive
from zipfile import ZipFile
import ffmpeg
from pymediainfo import MediaInfo
import mimetypes
# Mount Google Drive
drive.mount('/content/drive')
clear_output()


def deleteFolder(input_folder):
    !rm -rf "{input_folder}"
    print(f"Folder Deleted: {input_folder}")

def strToList(listStr, separator=","):
    listStr = ' '.join(listStr.split()) # Remove extra spaces from String
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    showAllURLs = "\n".join(f"{i+1}. {ele}" for i, ele in enumerate(myList))
    print(showAllURLs)
    return myList

def extract_file_info(file_path):
    # parent_dir_path = os.path.dirname(file_path)
    # file_name = os.path.basename(file_path)
    parent_dir_path, file_name = os.path.split(file_path)
    _name, _ext = os.path.splitext(file_name)
    return parent_dir_path,  file_name, _name, _ext[1:]

def getMKVFilesList(folder_path, relative=0):
    all_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.mkv'):
                full_path = os.path.join(root, file)
                if relative == 1:
                    full_path = os.path.relpath(full_path, folder_path)
                all_files.append(full_path)

    all_files.sort()
    return all_files


def extractArchive(Archive_File, extracted_folder=''):
    if not extracted_folder:
        extracted_folder = os.path.splitext(Archive_File)[0]
    if os.path.exists(extracted_folder):
        if Delete_Extracted_DIR:
            deleteFolder(extracted_folder)
        print(f"Archive File Already Exists:\n{output_file}")
        return extracted_folder

    os.makedirs(extracted_folder, exist_ok=True)
    print(f"\nExtracting archive to: {extracted_folder}")
    patoolib.extract_archive(Archive_File, outdir=extracted_folder)
    print(f"Extraction complete: {extracted_folder}\n")
    return extracted_folder


def runCommand(cmd, showLog=0):
    if showLog == 1:
        # Print logs in real-time, if 2nd param is 1
        print(f"Running command: {cmd}")
        process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        for line in iter(process.stdout.readline, b''):
            print(line.decode("utf-8"), end="")

        process.stdout.close()
        process.wait()
    else:
        process = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return process.returncode


def filter_Filename(filename):
    MAX_FILENAME_LENGTH = 245
    # Replace unsuitable characters with a hyphen
    invalid_chars = r'[<>:"/\\|?*\']'
    valid_filename = re.sub(invalid_chars, '-', filename)
    # Truncate the filename if it exceeds the maximum length
    if len(valid_filename) > MAX_FILENAME_LENGTH:
        # Preserve the file extension if present
        if '.' in valid_filename:
            name, extension = valid_filename.rsplit('.', 1)
            truncated_name = name[:MAX_FILENAME_LENGTH - len(extension) - 1]  # Reserve space for the dot
            valid_filename = f"{truncated_name}.{extension}"
        else:
            valid_filename = valid_filename[:MAX_FILENAME_LENGTH]

    return valid_filename


def getResponseFileName(url):
    response = requests.get(url, stream=True)

    # Extract the filename from the Content-Disposition header
    filename = response.headers.get('content-disposition', '')
    if 'filename=' in filename:
        filename = filename.split("filename=")[-1].strip('"')
        return filter_Filename(filename)
    return ""


def Aria2Downloader(url, output_file=''):
    if not url.startswith("http"):
        print(f"Invalid URL: {url}")
    _responseFileName = getResponseFileName(url)
    if _responseFileName:
        if not output_file:
            output_file = _responseFileName
        if os.path.exists(output_file):
            print(f"Download File Already Exists: {output_file}")
            return output_file
        else:
            """Download a file using Aria2."""
            print(f"Downloading file: {url}")
            command = f"aria2c -x 16 -s 16 -o '{output_file}' {url}"
            status_code = runCommand(command, 1)
            if status_code == 0:
                print(f"\nDownload Successfull: {output_file}\n")
                return output_file
            else:
                print(f"Download Error Status code({status_code}): {url}")
    else:
        print(f"Download Error Response FileName: {url}")
    return url



def mergeMKVs(video_paths, output_dir, max_size_gb):
    if len(video_paths) < 2:
        print(video_paths, f" ==> Can't Merge {len(video_paths)} video")
        return video_paths

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Base output file path
    base_output_path = os.path.join(output_dir, extract_file_info(video_paths[0])[2] + "_merged_")

    # Convert max_size_gb to bytes (mkvmerge uses bytes for splitting)
    max_size_bytes = max_size_gb * 1024 * 1024 * 1024

    command = [
        "mkvmerge",
        "-o", f"'{base_output_path}.mkv'",
        "--split", f"{max_size_bytes}",
    ]

    # Append valid videos sequentially
    for i, video_path in enumerate(video_paths):
        if i == 0:
            command.append(f"'{video_path}'")  # First video
        else:
            command.extend(["+", f"'{video_path}'"])  # Append subsequent videos

    # Run the mkvmerge command
    command_str = ' '.join(command)
    runCommand(command_str, 1)



def main(filesListStr, output_dir, max_size_gb):
    Files_List = strToList(filesListStr, ",")
    _File = Files_List[0]

    if len(Files_List) == 0 or (len(Files_List) == 1 and os.path.exists(_File) and _File.endswith('.mkv')):
        print(Files_List, " ==> Give At Least 2 files for Merging")
        exit()

    if len(Files_List) == 1:
        _File = Files_List[0]
        if _File.startswith('http'):
            _File = Aria2Downloader(_File)
        if os.path.exists(_File):
            if patoolib.is_archive(_File):
                _Folder = extractArchive(_File)
                Files_List = getMKVFilesList(_Folder)
            elif os.path.isdir(_File):
                Files_List = getMKVFilesList(_File)

    valid_Files_List = []
    if len(Files_List) > 1:
        for i, valid_File in enumerate(Files_List):
            if valid_File.startswith('http'):
                valid_File = Aria2Downloader(valid_File)
            if os.path.exists(valid_File):
                if valid_File.endswith('.mkv'):
                    valid_File = os.path.abspath(valid_File)
                    valid_Files_List.append(valid_File)
                else:
                    print("File Exists but Unsupported File Format: ", valid_File)
            else:
                print(f"Error FILES {i+1}: {valid_File}")
                exit()
        print("Valid Files are:\n", "\n".join(valid_Files_List))

    if len(valid_Files_List) > 1:
        if JUST_DOWNLOAD:
            exit()
        if not output_dir:
            output_dir = '/content/drive/MyDrive'
        mergeMKVs(valid_Files_List, output_dir, max_size_gb)

main(MKV_FILES, OUTPUT_DIR, MAX_FileSize)

In [ ]:


#@title FFmpeg

FILES = "/content/3"  # @param {type: "string"}
FFMPEG_OPTIONS = "-map 0:v -map 0:a -map 0:s? -c copy"  #@param {type: "string"}
Delete_Extracted_Data = False  #@param {type: "boolean"}
#@markdown ***Start 00:00:00 / End 00:00:10***
trim_start = ""  #@param {type: "string"}
trim_end = ""  #@param {type: "string"}
JUST_DOWNLOAD= False  #@param {type: "boolean"}
JUST_EXTRACT= False  #@param {type: "boolean"}

from IPython.display import clear_output
import subprocess
import os
import re
import requests
import patoolib
from google.colab import drive
from zipfile import ZipFile
from pymediainfo import MediaInfo
import mimetypes
# Mount Google Drive
drive.mount('/content/drive')
clear_output()


def strToList(listStr, separator=","):
    listStr = ' '.join(listStr.split()) # Remove extra spaces from String
    listStr = '\n'.join(listStr.split()) # Remove newLines from String
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    finalList = list(set(myList)) # Remove duplicates items from list
    showAllElements = "\n".join(f"{i+1} ==> {ele}" for i, ele in enumerate(finalList))
    print(showAllElements, "\n\n")
    return finalList


def printObj(obj):
    objToStr = str(vars(obj))
    print(objToStr)


def getSize(input_file):
    !du -h "{input_file}"
    !du -BM "{input_file}"


def copy(source, destination, isDeleteSource=0):
    if isDeleteSource == 0:
        !rsync -ah --progress "{source}" "{destination}"
    if isDeleteSource == 1:
        !rsync -ah --progress --remove-source-files "{source}" "{destination}"
        print(f"Source deleted: {source}")
    print(f"Transfer Successfull:\nSource => {source}\nDestination => {destination}")


def deleteFile(input_file):
    !rm "{input_file}"
    print(f"File Deleted: {input_file}")


def deleteFolder(input_folder):
    !rm -rf "{input_folder}"
    print(f"Folder Deleted: {input_folder}")


def printMediaInfo(input_file):
    """Display media information."""
    !mediainfo "{input_file}"


def extract_file_info(file_path):
    # parent_dir_path = os.path.dirname(file_path)
    # file_name = os.path.basename(file_path)
    parent_dir_path, file_name = os.path.split(file_path)
    _name, _ext = os.path.splitext(file_name)
    return parent_dir_path,  file_name, _name, _ext[1:]

def add_suffix_to_file(file_path, suffix="_Suffix"):
    parent_dir_path, file_name = os.path.split(file_path)
    name, ext = os.path.splitext(file_name)
    new_file_name = f"{name}{suffix}{ext}"
    new_file_path = os.path.join(parent_dir_path, new_file_name)
    return new_file_path

def filter_existing_filepath(base_path):
    directory, filename = os.path.split(base_path)
    name, ext = os.path.splitext(filename)

    counter = 1
    new_path = base_path
    while os.path.exists(new_path):
        new_name = f"{name} ({counter}){ext}" if ext else f"{name} ({counter})"
        new_path = os.path.join(directory, new_name)
        counter += 1

    return new_path


def getFilesList(folder_path, relative=0):
    all_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            full_path = os.path.join(root, file)
            if relative == 1:
                full_path = os.path.relpath(full_path, folder_path)
            all_files.append(full_path)

    all_files.sort()
    return all_files


def runCommand(cmd, showLog=0):
    if showLog == 1:
        # Print logs in real-time, if 2nd param is 1
        print(f"Running command: {cmd}")
        process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        for line in iter(process.stdout.readline, b''):
            print(line.decode("utf-8"), end="")

        process.stdout.close()
        process.wait()
    else:
        process = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return process.returncode


def filter_Filename(filename):
    MAX_FILENAME_LENGTH = 245
    # Replace unsuitable characters with a hyphen
    invalid_chars = r'[<>:"/\\|?*\']'
    valid_filename = re.sub(invalid_chars, '-', filename)
    # Truncate the filename if it exceeds the maximum length
    if len(valid_filename) > MAX_FILENAME_LENGTH:
        # Preserve the file extension if present
        if '.' in valid_filename:
            name, extension = valid_filename.rsplit('.', 1)
            truncated_name = name[:MAX_FILENAME_LENGTH - len(extension) - 1]  # Reserve space for the dot
            valid_filename = f"{truncated_name}.{extension}"
        else:
            valid_filename = valid_filename[:MAX_FILENAME_LENGTH]

    return valid_filename


def getResponseFileName(url):
    response = requests.get(url, stream=True)

    # Extract the filename from the Content-Disposition header
    filename = response.headers.get('content-disposition', '')
    if 'filename=' in filename:
        filename = filename.split("filename=")[-1].strip('"')
        return filter_Filename(filename)
    return ""


def Aria2Downloader(url, output_file=''):
    if not url.startswith("http"):
        print(f"Invalid URL: {url}")
    _responseFileName = getResponseFileName(url)
    if _responseFileName:
        if not output_file:
            output_file = _responseFileName
        if os.path.exists(output_file):
            print(f"Download File Already Exists: {output_file}")
            return output_file
        else:
            """Download a file using Aria2."""
            print(f"Downloading file: {url}")
            command = f"aria2c --continue=true --file-allocation=none -x 16 -s 16 -o \"{output_file}\" {url}"
            status_code = runCommand(command, 1)
            if status_code == 0:
                print(f"\nDownload Successfull: {output_file}\n")
                return output_file
            else:
                print(f"Download Error Status code({status_code}): {url}")
    else:
        print(f"Download Error Response FileName: {url}")
    return url


def useFFmpeg(input_file, output_file='', log=0):
    if not output_file:
        output_file = input_file
    output_file = filter_existing_filepath(output_file)
    global FFMPEG_OPTIONS
    global trim_start
    global trim_end
    trim = ""
    if trim_start:
        trim = f"-ss {trim_start} -to {trim_end}"
    if not FFMPEG_OPTIONS:
        FFMPEG_OPTIONS = "-map 0:v -map 0:a:m:language:hin -map 0:s? -c copy"
    command = f"ffmpeg -i \"{input_file}\" {trim} {FFMPEG_OPTIONS} -y \"{output_file}\""
    runCommand(command, log)
    print("Original Size:")
    getSize(input_file)
    print("NEW Size:")
    getSize(output_file)
    return output_file

def useFFmpegFolder(input_folder, log=0):
    ffmpeg_folder = input_folder + "_ffmpeg"
    if os.path.exists(ffmpeg_folder):
        deleteFolder(ffmpeg_folder)
    os.makedirs(ffmpeg_folder, exist_ok=True)

    for i, file in enumerate(getFilesList(input_folder)):
        if file.endswith(('.mkv', '.mp4', '.avi')):
            output_file = os.path.join(ffmpeg_folder, os.path.split(file)[1])
            print(f"{i+1}: {file}")
            useFFmpeg(file, output_file, log)

    print(f"\nFFmpeg Folder Operation Success: {ffmpeg_folder}\n")
    return ffmpeg_folder


def extractArchive(Archive_File, extracted_folder=''):
    if not extracted_folder:
        extracted_folder = os.path.splitext(Archive_File)[0]
    if os.path.exists(extracted_folder):
        deleteFolder(extracted_folder)
    os.makedirs(extracted_folder, exist_ok=True)
    print(f"\nExtracting archive to: {extracted_folder}")
    patoolib.extract_archive(Archive_File, outdir=extracted_folder)
    print(f"Extraction complete: {extracted_folder}\n")
    return extracted_folder

def createArchive(_folder, Archive_filepath):
    if os.path.exists(Archive_filepath):
        Archive_filepath = filter_existing_filepath(Archive_filepath)
    print("Building ZIP ===> ", Archive_filepath)
    patoolib.create_archive(Archive_filepath, getFilesList(_folder))
    print(f"Archived Success: {Archive_filepath}")
    return Archive_filepath

def processArchive(Archive_File):
    result_Archive = "/content/drive/MyDrive/" + os.path.split(Archive_File)[1]
    extracted_folder = extractArchive(Archive_File)
    if JUST_EXTRACT:
     return
    ffmpeg_folder = useFFmpegFolder(extracted_folder)
    createArchive(ffmpeg_folder, result_Archive)
    getSize(Archive_File)
    getSize(result_Archive)
    if Delete_Extracted_Data:
        deleteFolder(extracted_folder)
        deleteFolder(ffmpeg_folder)
    return result_Archive


# -------- Main Workflow --------

def main(filesListStr):
    drive_root = '/content/drive/MyDrive/'
    Files_List = strToList(filesListStr)

    for i, _File in enumerate(Files_List):
        if _File.startswith('http'):
            _File = Aria2Downloader(_File)
            if JUST_DOWNLOAD:
                return
        if os.path.exists(_File):
            if _File.endswith('.mkv'):
                output_file = ""
                if not _File.startswith(drive_root):
                    output_file = os.path.join(drive_root, os.path.split(_File)[1])
                useFFmpeg(_File, output_file, 1)
            elif patoolib.is_archive(_File):
                processArchive(_File)
            elif os.path.isdir(_File):
                useFFmpegFolder(_File, 1)
            else:
                print("File Exists but Unsupported File: ", _File)
        else:
            print(f"Error FILES {i+1}: {_File}")

main(FILES)

In [ ]:
#@title File Delete

Delete_File = "/content/3"  #@param {type: "string"}
!ls -lh "{Delete_File}"
!rm -rf "{Delete_File}"